# The sound of Silence

## Todo List
* É nessario resolver o problema para diversas faixas de onda de audio - Fixed
##
* modelo_um:
    O modelo foi treinado com apenas 5% do conjunto selecionado aleatoriamente, o cojunto de treinamento foi bastante resumido por limitação de hardware.
    Como esperado os resultados não foram nada satisfatorios, visto que o modelo consegue apenas classificar os as amostras como do tipo 1

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        import data
from keras.models import Sequential
from keras.layers import Dense,Dropout

## pastas
audios = "/home/joseildo/SpeechDetection/musan/"

data.iterateOnFold(audios, shuffle = True)

## modelo de teste
modelo_um = Sequential()
modelo_um.add(Dense(1600, input_shape=(None,1600)))
modelo_um.add(Dense(1600,activation="sigmoid"))
modelo_um.add(Dense(1600,activation="sigmoid"))
modelo_um.add(Dropout(0.2))
modelo_um.add(Dense(1600,activation="relu"))
modelo_um.add(Dense(4,activation = 'softmax'))
modelo_um.compile(loss='categorical_crossentropy',optimizer = 'sgd',metrics=['acc'])

while(data.hasNext()):
    x,y = data.next()
    modelo_um.fit(x,y,nb_epoch = 1)
    modelo_um.save_weights("/home/joseildo/SpeechDetection/pyHumanDetect/pounds3.h5")

In [1]:
import data
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras import metrics
## pastas
audios = "/home/joseildo/SpeechDetection/musan/"

data.iterateOnFold(audios, shuffle = True)

## modelo de teste
modelo_2 = Sequential()
modelo_2.add(Dense(1600, input_shape=(None,1600)))
modelo_2.add(Dense(1600,activation="sigmoid"))
modelo_2.add(Dense(4,activation = 'softmax'))
modelo_2.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=[metrics.categorical_accuracy])
modelo_2.save("model_three.h5")

while(data.hasNext()):
    x,y = data.next()
    modelo_2.fit(x,y,nb_epoch = 1)
    modelo_2.save_weights("/home/joseildo/SpeechDetection/pyHumanDetect/pounds_conv.h5")

Using TensorFlow backend.


TypeError: 'int' object is not callable

In [1]:
import data
from keras.models import Sequential
from keras.layers import Convolution1D,Dropout,Dense, BatchNormalization
from keras import metrics
## pastas
audios = "/home/joseildo/SpeechDetection/musan/"

data.iterateOnFold(audios, shuffle = True)

## modelo de teste
modelo_3 = Sequential()
modelo_3.add(BatchNormalization(input_shape=(None,1600)))
modelo_3.add(Convolution1D(100, 100))
modelo_3.add(Dense(100,activation="sigmoid"))
modelo_3.add(Dense(4,activation = 'softmax'))
modelo_3.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=[metrics.categorical_accuracy])
#modelo_3.save("model_three.h5")

while(data.hasNext()):
    x,y = data.next()
    modelo_3.fit(x,y,nb_epoch = 1)
    modelo_3.save_weights("/home/joseildo/SpeechDetection/pyHumanDetect/pounds_conv.h5")

Using TensorFlow backend.


[------------------------------------------------------------] 0.0% ...loading and running fold
Epoch 1/1


InvalidArgumentError: Incompatible shapes: [1,2502] vs. [1,2403]
	 [[Node: Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/cpu:0"](ArgMax, ArgMax_1)]]

Caused by op 'Equal', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/ioloop.py", line 887, in start
    handler_func(fd_ob

In [ ]:
from keras.models import load_model
import data
import os
model = load_model(os.getcwd()+"/model_one.h5")
model.load_weights(os.getcwd()+"/pounds.h5")
data.loadAudio(os.getcwd()+"/teste/2001.wav")
data.generateData()
x, y = data.prepareData()
result = model.predict(x)
